In [ ]:
import tensorflow as tf
import random
import numpy as np
import cv2
import glob
from tqdm import tqdm
from PIL import Image
from pathlib import Path
import pickle
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Dense, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard
import time

In [ ]:
training_data = []
DATADIR = "C:/Users/User/Desktop/ML/Clothes"
CATEGORIES = ["Notags", "Tags"]
IMG_SIZE = 256
def create_training_data():
    for category in CATEGORIES:  # notags and tags categories

        path = os.path.join(DATADIR,category)  # create path to notags and tags
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=nptag 1=tag

        for img in tqdm(os.listdir(path)):  # iterate over each image per notags and tags
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize image size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # pass errors
                pass

create_training_data() 

print(len(training_data))

In [ ]:
import random
#Shuffle data
random.shuffle(training_data)

In [ ]:
X = []
y = []
#Split to features and labels
for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)    

In [ ]:
#Normalize RGB from (0,255) to (0,1)
X = X/255.0

In [ ]:
#Store features array on disk
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
#Store labels array on disk
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
#Load features and labels without a need to extract and clean data
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()
pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
#Name for tensorboard logs My CNN 80-83% valaccuracy
NAME = "Tags-notags-clothes-CNN-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
IMG_SIZE = 256
# Model Architecture 
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('sigmoid'))



In [ ]:
#Model compilation
model.compile(loss="binary_crossentropy",
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])

model.summary()

In [ ]:
#Training with 15 epochs and 20% validation split. Logs of loss and accuracy are sent to tensorboard
epochs = 12
model.fit(X, y, batch_size=32, validation_split=0.2, epochs=epochs,verbose=1, callbacks=[tensorboard])

In [ ]:
#Save Model
model.save('5Conv-Tag-CNN.model')

In [ ]:
#Load Model
model = tf.keras.models.load_model('5Conv-Tag-CNN.model')

In [ ]:
import cv2
import tensorflow as tf

CATEGORIES = ["Notag", "Tag"]  # will use this to convert prediction num to string value

notags_test_filepath = "C:/Users/User/Desktop/ML/Clothes/Notags_test"
tags_test_filepath = "C:/Users/User/Desktop/ML/Clothes/Tags_test"

def prepare_testimages(filepath):
    IMG_SIZE = 256
    img_array = cv2.imread(filepath)  # read in the image
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)  # return the image with shaping that TF wants.

tag_count_right = 0        
tagcount = len(os.listdir(tags_test_filepath))
#Tags_test
for img in os.listdir(tags_test_filepath):
    prediction = model.predict([prepare_testimages(os.path.join(tags_test_filepath,img))])
    if int(prediction[0][0]) == 1:
        tag_count_right += 1

notag_count_right = 0        
notagcount = len(os.listdir(notags_test_filepath))

#Notags_test
for img in os.listdir(notags_test_filepath):
    prediction = model.predict([prepare_testimages(os.path.join(notags_test_filepath,img))])
    if int(prediction[0][0]) == 0:
        notag_count_right += 1
        
print("tag_accuracy:{0}, notag_accuracy:{1}".format(tag_count_right/tagcount, notag_count_right/notagcount))        

In [ ]:
import cv2
import tensorflow as tf

CATEGORIES = ["Notag", "Tag"]  # will use this to convert prediction num to string value

notags_test_filepath = "C:/Users/User/Desktop/ML/Clothes/Notags_test"
tags_test_filepath = "C:/Users/User/Desktop/ML/Clothes/Tags_test"

def prepare_testimages(filepath):
    IMG_SIZE = 256
    img_array = cv2.imread(filepath)  # read in the image
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)  # return the image with shaping that TF wants.

tag_count_right = 0        
tagcount = len(os.listdir(tags_test_filepath))
#Tags_test
for img in os.listdir(tags_test_filepath):
    prediction = model.predict([prepare_testimages(os.path.join(tags_test_filepath,img))])
    if int(prediction[0][0]) == 1:
        tag_count_right += 1

notag_count_right = 0        
notagcount = len(os.listdir(notags_test_filepath))

#Notags_test
for img in os.listdir(notags_test_filepath):
    prediction = model.predict([prepare_testimages(os.path.join(notags_test_filepath,img))])
    if int(prediction[0][0]) == 0:
        notag_count_right += 1
        
print("tag_accuracy:{0}, notag_accuracy:{1}".format(tag_count_right/tagcount, notag_count_right/notagcount))        